In [2]:
import pandas as pd
import geopandas as gpd
from gotrackit.map.Net import Net
from gotrackit.MapMatch import MapMatch
from gotrackit.model.Para import ParaGrid

In [3]:

if __name__ == '__main__':
    # 1.读取GPS数据
    gps_df = pd.read_csv(r'./product_data/gps_data.csv')
    # gps_df['time'] = pd.to_datetime(gps_df['time'], format='%Y/%m/%d %H:%M:%S')
    gps_df['time'] = gps_df['time'].str.replace('T', ' ').str.replace('Z', '').str.replace('/', '-')
    print(gps_df.head())
    # 2.构建一个net
    link = gpd.read_file(r'./product_data/roads/roads.shp')
    node = gpd.read_file(r'./product_data/nodes/nodes.shp')
    my_net = Net(link_gdf=link,
                 node_gdf=node,
                 fmm_cache=True, fmm_cache_fldr=r'./product_data', recalc_cache=True,
                 cut_off=1800.0,
                 not_conn_cost=800,
                 cache_slice=6)
    my_net.init_net()  # net初始化

    # 3. 匹配
    mpm = MapMatch(net=my_net, gps_buffer=1350, flag_name='xa_sample',
                   use_sub_net=True, use_heading_inf=True, omitted_l=5.0,
                   del_dwell=False, dwell_l_length=11.0, dwell_n=3,
                   export_html=False, export_geo_res=False, use_gps_source=False,
                   export_all_agents=False,
                   out_fldr=r'./product_data/match_results', dense_gps=False,
                   gps_radius=10.0, is_lower_f = False, dense_interval=140, beta=5, gps_sigma= 25,top_k=50)
    # 这里使用 multi_core_execute 方法
    # match_res, may_error_info, error_info = mpm.execute(gps_df=gps_df)
    match_res, may_error_info, error_info = mpm.multi_core_execute(gps_df=gps_df, core_num=12)
    # 输出结果
    print(match_res)
    match_res.to_csv(r'./product_data/match_results/match_result.csv', encoding='utf_8_sig', index=False)


         lng       lat                 time        type  agent_id  entity_id
0  104.10242  30.66200  2018-10-09 09:24:02  trajectory         1          0
1  104.10174  30.66395  2018-10-09 09:24:28  trajectory         1          0
2  104.10048  30.66709  2018-10-09 09:24:58  trajectory         1          0
3  104.10019  30.66779  2018-10-09 09:25:28  trajectory         1          0
4  104.10010  30.66799  2018-10-09 09:25:58  trajectory         1          0
a loop was detected in the line layer data, and it was automatically deleted...
__init__ costs :0.3504300117492676 seconds!
calc fmm cache...
fmm_path_cache costs :5.907142400741577 seconds!
using multiprocessing - 12 cores
         agent_id  seq  sub_seq                time loc_type  link_id  \
0               1    0        0 2018-10-09 09:24:02        s     3821   
1               1    0        1 2018-10-09 09:24:15        c      326   
2               1    1        0 2018-10-09 09:24:28        s     4101   
3               1    1